# Implementing a Key-Value Database

In this project, I will be using a B-Tree data structure as the building block for a fully functioning, saving-to-disk key-value store. The key-value store will inherit much of the behavior from the `BTree` class, but I will redesign some of the API to make it more user friendly, as well as add in some additional functionality.

## Node Class

In [1]:
class Node:
    def __init__(self, keys=None, children=None):
        self.keys = keys or []
        self.children = children or []

    def __repr__(self):
        return "<BNode: {}>".format(self.keys)    

    def is_leaf(self, node):
        node = Node()
        if len(self.children) == 0:
            return True
        else:
            return False

## BTree Class

In [2]:
class BTree:
    def __init__(self, t):
        self.t = t
        self.root = None

    def insert_non_full(self, node, key):
        if node.is_leaf():
            if len(node.keys) >= 2*self.t - 1:
                return
            index = 0
            for k in node.keys:
                if key > k: index += 1
                else: break
            node.keys.insert(index, key)
            return
        index = 0
        for k in node.keys:
            if key > k: index += 1
            else: break
        if len(node.children[index].keys) == 2*self.t - 1:
            left_node, right_node, new_key = self.split(node.children[index])
            node.keys.insert(index, new_key)
            node.children[index] = left_node
            node.children.insert(index+1, right_node)
            if key > new_key:
                index += 1
        self.insert_non_full(node.children[index], key)

    def split(self, node):
        left_node = Node(keys=node.keys[:len(node.keys)//2], children=node.children[:len(node.keys)//2+1])
        right_node =  Node(keys=node.keys[len(node.keys)//2:], children=node.children[len(node.keys)//2:])
        new_key = right_node.keys.pop(0)
        return left_node, right_node, new_key
    
    def insert(self, key):
        if not self.root:
            self.root = Node(keys=[key])
            return
        if len(self.root.keys) == 2*self.t - 1:
            old_root = self.root
            self.root = Node()
            left, right, new_key = self.split(old_root)
            self.root.keys.append(new_key)
            self.root.children.append(left)
            self.root.children.append(right)
        self.insert_non_full(self.root, key)
        
    def insert_multiple(self, keys):
        for key in keys:
            self.insert(key)
            
    def search(self, node, term):
        if not self.root:
            return False
        index = 0
        for key in node.keys:
            if key == term:
                return True
            elif term > key:
                index += 1
        if node.is_leaf():
            return False 
        return self.search(node.children[index], term)
    
    def greater_than(self, node, term, upper_bound=None, inclusive=False):
        if not self.root:
            return []
        index = 0
        values = []
        for key in node.keys:
            if upper_bound is not None:
                if inclusive and key == upper_bound:
                    values.append(key)
                if key >= upper_bound:
                    break
            if term > key:
                index += 1
                continue
            if inclusive and key == term:
                values.append(key)
            if key > term:
                values.append(key)
            if not node.is_leaf():
                values += self.greater_than(
                    node.children[index],
                    term,
                    upper_bound,
                    inclusive
                )
            index += 1
        if not node.is_leaf():
            values += self.greater_than(
                node.children[index],
                term,
                upper_bound,
                inclusive
            )
        return values
    
    def less_than(self, node, term, lower_bound=None, inclusive=False):
        if not self.root:
            return []
        index = 0
        values = []
        for key in node.keys:
            if lower_bound is not None:
                if inclusive and key == lower_bound:
                    values.append(key)
                if key < lower_bound:
                    index += 1
                    continue
            if inclusive and key == term:
                values.append(key)
            if key < term:
                values.append(key)
            if not node.is_leaf():
                values += self.less_than(
                    node.children[index],
                    term,
                    lower_bound,
                    inclusive
                )
            index += 1
        if not node.is_leaf() and key <= term:
            values += self.less_than(
                node.children[index],
                term,
                lower_bound,
                inclusive
            )
        return values

## NodeKey Class

In [3]:
class NodeKey:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.key == other.key
        return self.key == other
    
    def __gt__(self, other):
        if isinstance(other, self.__class__):
            return self.key > other.key
        return self.key > other
        
    def __lt__(self, other):
        if isinstance(other, self.__class__):
            return self.key < other.key
        return self.key < other
        
    def __ge__(self, other):
        if isinstance(other, self.__class__):
            return self.key >= other.key
        return self.key >= other
        
    def __le__(self, other):
        if isinstance(other, self.__class__):
            return self.key <= other.key
        return self.key <= other

## DQKV Class

I will start by adding a `set()` and `get()` method to the DQKV ("DataQuest Key-Value") class. 

The `set()` method, similarly to that of a Python dictionary, will set a `value` for a given `key`, but only if the value is not `None`. If the value is `None`, an error will appear.

The `get()` method will retrieve a value from a given `key`. If the given `key` does not exist, an error will appear. 

I will then override the `__init__()` method, but also call the parent `__init__()` method using `super().__init__()`.

The `greater_than()` and `less_than()` methods from `BTree` can be a bit tedious to work with for someone who wants to just do a simple range query on their data. So, I will implement a new range query method that allows a more simple user interface, but still uses the old range queries on the backend. Here are a few examples of how the method will behave:

```
dqkv.range_query(0, 5, inclusive=True)
```
returns all values that are between 0 and 5, inclusive.

```
dqkv.range_query(5, None)
```
returns all values that are greater than 5.

```
dqkv.range_query(None, 5, inclusive=True)
```
returns all values that are less than 5.

The next method I will implement is `dump()`, which will save the file using the pickle module. Then, I will create a load `load()` function. This will take in a filename and assign a loaded object to an instance using a static method as opposed to an instance method. 

Lastly, I will create a `load_from_dict()` method that loads in keys and values from a Python dictionary, using the `set()` method that I previously defined. 

In [4]:
from btree import NodeKey, Node, BTree
import pickle

class DQKV(BTree):
    def __init__(self, type_, values=None):
        self.type = type_
        super().__init__(t=10)
    
    def get(self, key):
        value = self.search(self.root, key)
        if value is None:
            raise KeyError('Key {} does not exist'.format(key))
        return value
        
    def set(self, key, value):
        if value is None:
            raise ValueError('Cannot store None values')
        if not isinstance(key, self.type):
            raise KeyError('Key must be {} type'.format(self.type))
        if self.search(self.root, key) is not None:
            raise ValueError('Duplicate key value')
            
    def range_query(self, start, end, inclusive=False):
        if start is None:
            return self.less_than(node=self.root, term=end, lower_bound=None, inclusive=inclusive)
        elif end is None:
            return self.greater_than(node=self.root, term=start, upper_bound=None, inclusive=inclusive)
        else:
            return self.less_than(node=self.root, term=end, lower_bound=start, inclusive=inclusive)
        
    def dump(self, filename):
        filename = filename + '.dqdb'
        with open(filename, 'wb') as file:
            pickle.dump(self, file)
            return True
        return False
    
    def load_from_dict(self, dictionary):
        for key, value in dictionary.items():
            self.set(key, value)
    
    @staticmethod
    def load(filename):
        filename = filename + '.dqdb'
        with open(filename, 'rb') as file:
            return pickle.load(file)
        

I will now test a sample to ensure that my key-value store class is working as expected.

In [ ]:
dq = DQKV(int)
dq.set(1, 'this')
dq.set(2, 'is')
dq.set(3, 'a')
dq.set(4, 'test')

print(dq.range_query(1,3))

dq.dump('sample_store')
dqkv = DQKV.load('sample_store')

print(dqkv.range_query(1,3))
additional_keys = {
    5: 'for',
    6: 'my',
    7: 'kv store'
}
dqkv.load_from_dict(additional_keys)
print(dqkv.range_query(4,8))
print(dqkv.get(3))